# Singualar Value Thresholding / Nuclear norm relaxation
Results: (after around 10 iterations)
- Score: xx on Kaggle - did not publish as the method does not converge to a sensible RMSE
- For many choice of parameters $\tau$ and $\eta$, the RMSE is ~15 on training set

## Packages

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from utils import import_data_to_matrix, extract_submission
from utils import NUMBER_OF_MOVIES, NUMBER_OF_USERS
from utils import zscore_masked_items

## Data Preprocessings
- Extract data to row-column format
- Impute missing data with 0
- Normalize item by item

- Rating matrix A

In [2]:
A = import_data_to_matrix()

- Observation matrix Ω
- Normalize item by item (z-scores)
$$A_{ij} = \frac{A_{ij} - \overline{A{j}}}{std(A_{j})}$$

Important: 
- Only observed entries are updated
- Mean and std is computed only over observed entries

In [3]:
W = (A > 0).astype(int)
norm_A, mean_A, stddev_A = zscore_masked_items(A, W)

## Projected gradient descent
Let $A = U \text{diag}(\sigma_{i}) V^{\top}$, define
$$ \text{shrink}_{\tau}(A) = U \text{diag}(\sigma_{i} - \tau)_{+} V^{\top}$$
The algorithm is:
$$A^0 = 0$$
$$A^{t+1} = A^t + \eta\Pi_{\Omega}(A - \text{shrink}_{\tau}(A^t))$$

In [43]:
def rmse(A, A_t, W):
    return np.sum(W*((A-A_t)**2)) / np.sum(W)

def shrink(tau, A):
    U, s, Vt = np.linalg.svd(A, full_matrices=False)
    print(s[:10])
    s = s - tau
    print("s-tau", s[:10])
    s[s < 0] = 0 #clip singular values
    print("s clipped", s[:10])
    return np.dot(U * s, Vt)

A_t = np.zeros((NUMBER_OF_USERS , NUMBER_OF_MOVIES))
eta = 0.15
tau = 2000
shrink_tau_A_t = None
for epoch in tqdm(range(50)):
    shrink_tau_A_t = shrink(tau, A_t)
    A_t = A_t + eta * W * (A - shrink_tau_A_t)
    #iter+=1
    #eta = eta / (iter)**(1/2)
    print("k = ", k, ", RMSE = ", rmse(norm_A, shrink_tau_A_t, W))

  0%|          | 0/50 [00:00<?, ?it/s]

setting tau  =  0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
s-tau [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
s clipped [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


  2%|▏         | 1/50 [00:03<02:27,  3.02s/it]

k =  5 , RMSE =  1.0000000000000009
setting tau  =  50.858045365977596
[323.22755713  70.55344563  62.00729893  50.85804537  46.32002343
  43.53003662  42.00005233  39.06095707  38.2954933   37.1409861 ]
s-tau [272.36951176  19.69540026  11.14925357   0.          -4.53802194
  -7.32800875  -8.85799304 -11.79708829 -12.56255207 -13.71705927]
s clipped [272.36951176  19.69540026  11.14925357   0.           0.
   0.           0.           0.           0.           0.        ]


  4%|▍         | 2/50 [00:06<02:33,  3.19s/it]

...

k =  5 , RMSE =  1.677792600723686
setting tau  =  371.44741281667535
[2233.35935903  508.3037841   446.72811701  371.44741282  338.21234289
  316.40709287  305.84733289  282.82243755  274.33897817  262.164242  ]
s-tau [1861.91194621  136.85637128   75.28070419    0.          -33.23506992
  -55.04031995  -65.60007993  -88.62497527  -97.10843465 -109.28317081]
s clipped [1861.91194621  136.85637128   75.28070419    0.            0.
    0.            0.            0.            0.            0.        ]


 18%|█▊        | 9/50 [00:28<02:10,  3.17s/it]

k =  5 , RMSE =  1.8596002193572043


 18%|█▊        | 9/50 [00:31<02:24,  3.52s/it]


KeyboardInterrupt: 

- Undo the normalization.

In [15]:
print(shrink_tau_A_t)

[[0.42756985 0.76159923 0.99975634 ... 0.5869903  0.78252946 1.43644687]
 [1.64463584 2.1547667  2.64260856 ... 1.90404985 2.39352381 3.42246276]
 [0.7796484  1.3228193  1.72065146 ... 1.04024146 1.37444825 2.44037028]
 ...
 [0.66568373 1.045077   1.33811468 ... 0.84965006 1.10639549 1.85461678]
 [0.90673158 1.48481145 1.91784713 ... 1.1853105  1.55581064 2.69258824]
 [1.42402051 1.97821391 2.46279663 ... 1.70000936 2.1619646  3.26870618]]


In [16]:
rec_A = A_t
#undo normalization
for j in range(1000):
    rec_A[:,j] *= stddev_A[j]
    rec_A[:,j] += mean_A[j]

In [19]:
print(rec_A, np.min(rec_A), np.max(rec_A), np.mean(rec_A), np.median(rec_A))

[[  3.37941176   3.50094162   3.48358586 ...   3.23940678   3.3539823
    3.68230563]
 [  3.37941176   3.50094162   3.48358586 ... 113.33956435  44.60171118
   21.87030508]
 [  3.37941176   3.50094162   3.48358586 ...   3.23940678   3.3539823
    3.68230563]
 ...
 [  3.37941176   3.50094162   3.48358586 ...   3.23940678   3.3539823
    3.68230563]
 [  3.37941176   3.50094162   3.48358586 ...   3.23940678   3.3539823
    3.68230563]
 [  3.37941176   3.50094162   3.48358586 ...   3.23940678   3.3539823
   26.00490399]] -92.36440411063332 200.6733674310879 7.437268025784767 3.6412143514259427


## Export Predictions


In [23]:
extract_submission(rec_A, file="baseline")

In [27]:
temp = np.ones((4,5))
s = np.array([1,2,3,4,5])
S = np.diag(s)
print(temp @ S)
print(np.sum(norm_A**2)/np.sum(W))

[[1. 2. 3. 4. 5.]
 [1. 2. 3. 4. 5.]
 [1. 2. 3. 4. 5.]
 [1. 2. 3. 4. 5.]]
1.0000000000000009
